In [ ]:
!pip install scikit-learn==1.1.1 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.0.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.1.1 which is incompatible.


In [ ]:
!pip install mglearn
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn. linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

data, target = load_boston(return_X_y=True)

estimators = [('el', ElasticNet()), ('dt', DecisionTreeRegressor()), ('br', BayesianRidge())]
modelClf = StackingRegressor(estimators=estimators, final_estimator=SVR())

X_train, X_valid, y_train, y_valid = train_test_split(data, target, test_size=0.3, random_state=12)

modelClf.fit(X_train, y_train)
print(modelClf.score(X_valid, y_valid))

0.7502007312386907


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

----------2 task: **Bagging**---------------

In [ ]:
from sklearn.ensemble import BaggingRegressor
modelClf1 = BaggingRegressor(base_estimator=ElasticNet(alpha=0.001), n_estimators=50, random_state=12)
modelClf2= BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=50, random_state=12)
modelClf3 = BaggingRegressor(base_estimator=BayesianRidge(), n_estimators=50, random_state=12)

X_train, X_valid, y_train, y_valid = train_test_split(data, target, test_size=0.3, random_state=12)


modelClf1.fit(X_train, y_train)
modelClf2.fit(X_train, y_train)
modelClf3.fit(X_train, y_train)
print("Elastic:", modelClf1.score(X_valid, y_valid),"Tree", modelClf2.score(X_valid, y_valid),"Bayes", modelClf3.score(X_valid, y_valid) )

Elastic: 0.7026374613902243 Tree 0.8574167885843955 Bayes 0.68833696098105


-----3 task: **Optuna**----------

In [ ]:
!pip install optuna

In [ ]:
import numpy as np
import optuna

In [ ]:
alpha = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
l1_ratio = np.arange(0, 1, 0.01)
X_train, X_valid, y_train, y_valid = train_test_split(data, target, random_state=0)
def objective(trial):
  alpha = trial.suggest_float("alpha", 1e-5, 100.0, log=True)
  l1_ratio = trial.suggest_float("l1_ratio", 0, 1)
  clf=ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=0)
  clf.fit(X_train, y_train)
  return clf.score(X_valid, y_valid)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2024-03-28 12:38:39,344] A new study created in memory with name: no-name-7533330a-968f-44fb-b3ee-cc9f19d0c915
[I 2024-03-28 12:38:39,364] Trial 0 finished with value: 0.5633382727914129 and parameters: {'alpha': 0.9805346989803064, 'l1_ratio': 0.275319852242354}. Best is trial 0 with value: 0.5633382727914129.
[I 2024-03-28 12:38:39,382] Trial 1 finished with value: 0.6282971463926051 and parameters: {'alpha': 0.0027514003609984644, 'l1_ratio': 0.32366820042933}. Best is trial 1 with value: 0.6282971463926051.
[I 2024-03-28 12:38:39,393] Trial 2 finished with value: 0.6351654614043919 and parameters: {'alpha': 8.883144765045016e-05, 'l1_ratio': 0.2787944802998875}. Best is trial 2 with value: 0.6351654614043919.
[I 2024-03-28 12:38:39,402] Trial 3 finished with value: 0.1856141857120016 and parameters: {'alpha': 93.37591549277636, 'l1_ratio': 0.2281548714305469}. Best is trial 2 with value: 0.6351654614043919.
[I 2024-03-28 12:38:39,408] Trial 4 finished with value: 0.5129879038920

------4 task: **Хи-квадрат Пирсона**-------------

In [ ]:
import pandas as pd
df = pd.DataFrame({'good': [20, 43, 10], 'norm': [8, 20, 40], 'bad': [2, 17, 10]})
print(df)

   good  norm  bad
0    20     8    2
1    43    20   17
2    10    40   10


In [ ]:
df1 = pd.DataFrame({'good': [20, 43, 10], 'norm': [8, 20, 40], 'bad': [2, 17, 10]})
df1['sum']=df1.sum(axis=1)
df1.loc[len(df1.index)]=df1.sum(axis=0)/np.sum(df1['sum'])
print(df1)

        good  norm        bad   sum
0  20.000000   8.0   2.000000  30.0
1  43.000000  20.0  17.000000  80.0
2  10.000000  40.0  10.000000  60.0
3   0.429412   0.4   0.170588   1.0


In [ ]:
print(np.sum(df1['sum']))

170


In [ ]:
print(df1.loc[[0]])

   good  norm  bad   sum
0  20.0   8.0  2.0  30.0


In [ ]:
df3 = pd.DataFrame({'good': [20, 43, 10], 'norm': [8, 20, 40], 'bad': [2, 17, 10]})

In [ ]:
for i in ['good', 'norm', 'bad']:
  for j in range(3):
    df3[i][j]=df1[i][3]*df1['sum'][j]
print(df3)

        good  norm        bad
0  12.882353    12   5.117647
1  34.352941    32  13.647059
2  25.764706    24  10.235294


<ipython-input-85-1bcb0ef3ccfa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[i][j]=df1[i][3]*df1['sum'][j]


In [ ]:
df4=df-df3
df4

,good,norm,bad
0,7.117647,-4,-3.117647
1,8.647059,-12,3.352941
2,-15.764706,16,-0.235294


In [ ]:
df4=np.square(df4)
df4

,good,norm,bad
0,50.660900,16,9.719723
1,74.771626,144,11.242215
2,248.525952,256,0.055363


In [ ]:
df5=df4/df3
df5

,good,norm,bad
0,3.932581,1.333333,1.899256
1,2.176571,4.500000,0.823783
2,9.645984,10.666667,0.005409


In [ ]:
print(np.sum(df5.sum(axis=0)))

good    15.755137
norm    16.500000
bad      2.728448
dtype: float64
